Import all the necessary Python packages, the stellar, and planet sample.

In [1]:
import ExoMult
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


stellarKepler=pd.read_csv("Kepler_stellar.csv",sep=",", engine='python', header=0)
planetKepler=pd.read_csv("Kepler_planet.csv",sep=",", engine='python', header=0)

stellarK2=pd.read_csv("K2_stellar.csv",sep=",", engine='python', header=0)
planetK2=pd.read_csv("K2_planet.csv",sep=",", engine='python', header=0)

####Extract Super-Earths and Sub-Neptunes
planetK2SubN=planetK2[(planetK2.Rad>10**(-0.11*np.log10(planetK2.Period) + 0.36))&(planetK2.Rad<4)]
planetK2SupE=planetK2[(planetK2.Rad<10**(-0.11*np.log10(planetK2.Period) + 0.36))&(planetK2.Rad>1)]

planetKeplerSubN=planetKepler[(planetKepler.Rad>10**(-0.11*np.log10(planetKepler.Period) + 0.36))&(planetKepler.Rad<4)]
planetKeplerSupE=planetKepler[(planetKepler.Rad<10**(-0.11*np.log10(planetKepler.Period) + 0.36))&(planetKepler.Rad>1)]


The next few step just clean up the data frames, improving the speed of the code. 

First, we amplfy the stellar sample by a factor (sampleMult). This ensures we are getting an acurate measurement of occurrence. Smaller samples are prone to changes on any given draw with foward modeling as expected by Poisson statisitics. Multiplying the sample by some factor acts as a way of running the simulation numerous times to measure the median occurrence quickly. The downside is that we may lose information about the posterior distribtuion in doing so. As an alternative, you could set sampleMult to 1 and run the simulation numerous times to get an empircal sampling of the posterior. 

In [2]:
####Multiply the stellar sample to reduce convergence length
sampleMult=10

stellarK2=(pd.concat([stellarK2]*sampleMult, ignore_index=True))
stellarKepler=(pd.concat([stellarKepler]*sampleMult, ignore_index=True))


In [3]:

#Clean up the data frame so you aren't dragging a ton of unimportant columns along

stellarK2=stellarK2.drop(columns=['k2stars2_oid','k2_ra', 'k2_dec', 'k2_kepmag', 'k2_vjmag', 'k2_kmag','random_index', 'ra', 'dec', 'parallax', 'pmra', 'pmdec',                                               
       'astrometric_params_solved', 'ipd_frac_multi_peak', 'ipd_frac_odd_win',                                 
       'l', 'b', 'non_single_star', 'angdist', 'scaleHeigth_error','U_Rad', 'u_Rad', 'U_Teff', 'U_logg','U_FeH','U_Mass',
       'CDPP1_5', 'CDPP2', 'CDPP2_5', 'CDPP3', 'CDPP4', 'CDPP5', 'k2_campaign', 'd_mag',
          'CDPP6', 'CDPP7', 'CDPP8', 'CDPP9','RUWE', 'Ncomp','source_id', 'scaleHeigth'])
stellarKepler=stellarKepler.drop(columns=['U_Teff', 'u_Teff',
              'U_Mass', 'u_Mass', 'U_Rad', 'u_Rad', 'iso_age','iso_age_err1', 'iso_age_err2',
              'gmag', 'e_gmag', 'Ksmag', 'e_Ksmag', 'Par', 'e_Par', '[Fe/H]', 'e_[Fe/H]','ra_kic', 'dec_kic', 'source_id', 'random_index', 'ra', 'dec', 'parallax', 'pmra', 'pmdec', 'astrometric_params_solved', 'ipd_frac_multi_peak', 'ipd_frac_odd_win', 'rv_nb_transits', 'l', 'b', 'non_single_star', 'kepler_gaia_ang_dist', 'kepmag', 'teff', 'logg_err1', 'logg_err2', 'feh', 'feh_err1', 'feh_err2',
              'radius', 'radius_err1', 'radius_err2', 'mass', 'mass_err1', 'mass_err2', 'nconfp', 'nkoi', 'ntce', 'jmag', 'hmag', 'kmag',
              'U_FeH', 'U_logg','scaleHeigth_error','CDPP2',
              'CDPP2_5', 'CDPP3', 'CDPP3_5', 'CDPP4_5', 'CDPP5', 'CDPP6', 'CDPP7_5',
              'CDPP9', 'CDPP10_5', 'CDPP12', 'CDPP12_5', 'scaleHeigth','Ncomp', 'teff_err1', 'teff_err2'])

### down-casting here to improve the speed

fcols = stellarK2.select_dtypes('float').columns
icols = stellarK2.select_dtypes('integer').columns

stellarK2[fcols] = stellarK2[fcols].apply(pd.to_numeric, downcast='float')
stellarK2[icols] = stellarK2[icols].apply(pd.to_numeric, downcast='integer')

fcols = stellarKepler.select_dtypes('float').columns
icols = stellarKepler.select_dtypes('integer').columns

stellarKepler[fcols] = stellarKepler[fcols].apply(pd.to_numeric, downcast='float')
stellarKepler[icols] = stellarKepler[icols].apply(pd.to_numeric, downcast='integer')


Next, define a few functions that calculate the  un-normalized completeness for the Super-Earths, Sub-Neptunes, and a more general sample. 

un-norm completeness? ....In other words, we calculate the number of planets one expects to find if every star has a single planet.

In [4]:
def superEarthCompleteness(stellarInput, mission, sampleMult,  pMin,pMax=None,rMin=1, rMax=4, alpha=0, beta1=0, beta2=0, pbreak=None):
    

    stellarDumb=stellarInput*1
    
    #### assign one planet to each star with a given broken power-law distribution
    stellarDumb['m']=1
    stellarDumb['per_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=beta1,bCon=beta2, xmix=pbreak, xmin=pMin,xmax=pMax)
    stellarDumb['rad_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=alpha,xmax=10**(-0.11*np.log10(stellarDumb.per_m1) + 0.37),xmin=rMin)

    
    params = ExoMult.PopParams(stellar=stellarDumb, mission=mission)
    outputPopulation=ExoMult.SimulateDetection(params)
    
    #### remove non-detections
    outputPopulation=outputPopulation[outputPopulation.rad_m1>0]
    
    ### Calculate Completeness
    complete=len(outputPopulation)/sampleMult
    
    return(complete)

def subNeptuneCompleteness(stellarInput, mission, sampleMult, pMin,pMax=None,rMin=1, rMax=4, alpha=0, beta1=0, beta2=0, pbreak=None):
    

    stellarDumb=stellarInput*1
    
    #### assign one planet to each star with a given broken power-law distribution
    stellarDumb['m']=1
    stellarDumb['per_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=beta1,bCon=beta2, xmix=pbreak, xmin=pMin,xmax=pMax)
    stellarDumb['rad_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=alpha,xmin=10**(-0.11*np.log10(stellarDumb.per_m1) + 0.37),xmax=rMax)

    
    params = ExoMult.PopParams(stellar=stellarDumb, mission=mission)
    outputPopulation=ExoMult.SimulateDetection(params)
    
    #### remove non-detections
    outputPopulation=outputPopulation[outputPopulation.rad_m1>0]
    
    ### Calculate Completeness
    complete=len(outputPopulation)/sampleMult
    
    return(complete)


def generalCompleteness(stellarInput, mission, sampleMult, pMin,pMax=None,rMin=1, rMax=20, alpha=0, beta1=0, beta2=0, pbreak=None):
    

    stellarDumb=stellarInput*1
    
    #### assign one planet to each star with a given broken power-law distribution
    stellarDumb['m']=1
    stellarDumb['per_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=beta1,bCon=beta2, xmix=pbreak, xmin=pMin,xmax=pMax)
    stellarDumb['rad_m1']=ExoMult.brokenPow(n=len(stellarDumb),
                        aCon=alpha,xmin=rMin,xmax=rMax)

    
    params = ExoMult.PopParams(stellar=stellarDumb, mission=mission)
    outputPopulation=ExoMult.SimulateDetection(params)
    
    #### remove non-detections
    outputPopulation=outputPopulation[outputPopulation.rad_m1>0]
    
    ### Calculate Completeness
    complete=len(outputPopulation)/sampleMult
    
    return(complete)

Create occurrence plots as a function of stellar temp.

Note: As highlighted previously, the uncertainty of the model is lost in using a sampleMult greater than one. We rely on the Possion statisitics to for uncertainty (i.e., Sqrt(N)). In most cases, this is sufficiant. If the sample (or observed number of planets) is very small, this may be in invalid. In those cases, you need to measure the variable "complete" a bunch of times and look at how the output "occurRate" changes under this range of simulations. This can be computationally expensive, thus, I have ignored such cases here.

Also, it should be noted that each bin assumes a uniform radius (alpha=0) and period distribution (beta1=0, beta2=0) across the relavent parameter space (rMin=1R_Earth, rMax=4R_Earth, pMin=1day, pMax=40day). This is simple, but not accurate. In some cases, a more emprically driven radius and period distribution can change the output occurrence (the Zink et al. 2023 Table 1 values can be incorporated in such cases). However, this simple model works failry pretty well to first order.

In [5]:
from matplotlib.backends.backend_pdf import PdfPages

##Define number of bins
numBins=10

##Define range of occurrence
teffLow=3200 #K
teffHigh=6500 #K


plt.style.use('classic')
with PdfPages('SupEar_Teff_Occurrence.pdf')as pdf:
    plt.rcParams.update({'font.size': 14})
    plt.figure(figsize=(5,5))

    ###Assign memory to various values
    occurRate=np.zeros(numBins)
    occurRateerr=np.zeros(numBins)
    occurLimit=np.ones(numBins)*1e-10
    tempTick=np.zeros(numBins)
    bins=((teffHigh)-(teffLow))/len(occurRate)

    
    ##Define reliability for each planet in K2
    reli=np.ones(len(planetK2SupE))
    reli=np.where(planetK2SupE.Period<5,.99,reli)
    reli=np.where((planetK2SupE.Period>=5) & (planetK2SupE.Period<10),.94,reli)
    reli=np.where((planetK2SupE.Period>=10) & (planetK2SupE.Period<20),.86,reli)
    reli=np.where((planetK2SupE.Period>=20) & (planetK2SupE.Period<30),.87,reli)
    reli=np.where((planetK2SupE.Period>=30) & (planetK2SupE.Period<40),.75,reli)
    planetK2SupE["reli"]=reli*1
    

    for i in range(len(occurRate)):
        x1=i*bins+(teffLow)
        x2=(i+1)*bins+(teffLow)
        tempTick[i]=((i+.5)*bins+(teffLow))
        
        ##calculate K2 (un-norm) completeness for given bin
        complete=superEarthCompleteness(stellarInput=stellarK2[(stellarK2.Teff>=x1)&(stellarK2.Teff<=x2)],
                mission="K2", sampleMult=sampleMult,  pMin=1, pMax=40, pbreak=40, rMin=1, rMax=4, alpha=0, beta1=0, beta2=0)
        
        ##calculate and add the Kepler (un-norm) completeness for given bin
        complete+=superEarthCompleteness(stellarInput=stellarKepler[(stellarKepler.Teff>=x1)&(stellarKepler.Teff<=x2)],
                mission="Kepler", sampleMult=sampleMult,  pMin=1, pMax=40, pbreak=40, rMin=1, rMax=4, alpha=0, beta1=0, beta2=0)
        
        
        ####This just avoids singularities
        if complete==0:
            complete=1
            
        #### Sum number of real planets (both Kepler and K2) in a given bin (multiplied by their reliability)
        sumReli=np.sum(planetK2SupE.reli[(planetK2SupE.Teff<=x2)&(planetK2SupE.Teff>=x1)])+len(planetKeplerSupE[(planetKeplerSupE.Teff<=x2)&(planetKeplerSupE.Teff>=x1)])
 
        #Calculate occurrence and uncertainty
        occurRate[i]=sumReli/complete
        occurRateerr[i]=np.sqrt(sumReli)/complete
        
        #Calculate upper limits if needed
        if occurRate[i]==0:
            occurLimit[i]=4/complete
            occurRate[i]=1e-10
            occurRateerr[i]=1e-10
        elif occurRate[i]<=occurRateerr[i]:
            occurLimit[i]=occurRate[i]+3*occurRateerr[i]
            occurRate[i]=1e-10
            occurRateerr[i]=1e-10
            
    #Plot occurrence estimates and upper limits (3-sigma)
    plt.scatter(tempTick,occurLimit, color="#40A389", marker="v", zorder=100, facecolors="w", s=40)
    plt.errorbar(tempTick,occurRate,yerr=occurRateerr,color="#40A389", markeredgecolor="#40A389", fmt="o", zorder=101, markerfacecolor="w", markeredgewidth=1)

    
    #Plot features
    plt.yscale("log")
    plt.ylim([3e-2,3])
    plt.xlim([teffLow,teffHigh])
    plt.yticks([1e-1,1], ["10","100"]) 
    plt.xticks([3500,4500,5500,6500]) 
    plt.xlabel("Effective Stellar Temp. [K]")
    plt.ylabel("Planets per 100 Stars")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

/var/folders/bb/byt860fx2d1_xjx67wpqq_wr0000gn/T/ipykernel_33813/3913710534.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planetK2SupE["reli"]=reli*1


Now run it for the Sub-Neptunes

In [6]:
numBins=10

teffLow=3200
teffHigh=6500


plt.style.use('classic')
with PdfPages('SubNep_Teff_Occurrence.pdf')as pdf:
    plt.rcParams.update({'font.size': 14})
    plt.figure(figsize=(5,5))

    
    occurRate=np.zeros(numBins)
    occurRateerr=np.zeros(numBins)
    occurLimit=np.ones(numBins)*1e-10
    tempTick=np.zeros(numBins)
    bins=((teffHigh)-(teffLow))/len(occurRate)

    
    ##define reliability for each planet in K2
    reli=np.ones(len(planetK2SubN))
    reli=np.where(planetK2SubN.Period<5,.99,reli)
    reli=np.where((planetK2SubN.Period>=5) & (planetK2SubN.Period<10),.94,reli)
    reli=np.where((planetK2SubN.Period>=10) & (planetK2SubN.Period<20),.86,reli)
    reli=np.where((planetK2SubN.Period>=20) & (planetK2SubN.Period<30),.87,reli)
    reli=np.where((planetK2SubN.Period>=30) & (planetK2SubN.Period<40),.75,reli)
    planetK2SubN["reli"]=reli*1
    

    for i in range(len(occurRate)):
        x1=i*bins+(teffLow)
        x2=(i+1)*bins+(teffLow)
        tempTick[i]=((i+.5)*bins+(teffLow))
        
        ##calculate K2 (un-norm) completeness for given bin
        complete=subNeptuneCompleteness(stellarInput=stellarK2[(stellarK2.Teff>=x1)&(stellarK2.Teff<=x2)],
                mission="K2", sampleMult=sampleMult,  pMin=1, pMax=40, pbreak=40, rMin=1, rMax=4, alpha=0, beta1=0, beta2=0)
        
        ##calculate and add the Kepler (un-norm) completeness for given bin
        complete+=subNeptuneCompleteness(stellarInput=stellarKepler[(stellarKepler.Teff>=x1)&(stellarKepler.Teff<=x2)],
                mission="Kepler", sampleMult=sampleMult,  pMin=1, pMax=40, pbreak=40, rMin=1, rMax=4, alpha=0, beta1=0, beta2=0)
        
        
        ####This just avoids singularities
        if complete==0:
            complete=1
            
        #### Sum number of real planets (both Kepler and K2) in a given bin (multiplied by their reliability)
        sumReli=np.sum(planetK2SubN.reli[(planetK2SubN.Teff<=x2)&(planetK2SubN.Teff>=x1)])+len(planetKeplerSubN[(planetKeplerSubN.Teff<=x2)&(planetKeplerSubN.Teff>=x1)])
 
        #Calculate occurrence and uncertainty
        occurRate[i]=sumReli/complete
        occurRateerr[i]=np.sqrt(sumReli)/complete
        
        #Calculate Upper limits if needed
        if occurRate[i]==0:
            occurLimit[i]=4/complete
            occurRate[i]=1e-10
            occurRateerr[i]=1e-10
        elif occurRate[i]<=occurRateerr[i]:
            occurLimit[i]=occurRate[i]+3*occurRateerr[i]
            occurRate[i]=1e-10
            occurRateerr[i]=1e-10
            
    #plot occurrence estimates and upper limits (3-sigma)
    plt.scatter(tempTick,occurLimit, color="#40A389", marker="v", zorder=100, facecolors="w", s=40)
    plt.errorbar(tempTick,occurRate,yerr=occurRateerr,color="#40A389", markeredgecolor="#40A389", fmt="o", zorder=101, markerfacecolor="w", markeredgewidth=1)

    
    #Plot features
    plt.yscale("log")
    plt.ylim([3e-2,3])
    plt.xlim([teffLow,teffHigh])
    plt.yticks([1e-1,1], ["10","100"]) 
    plt.xticks([3500,4500,5500,6500]) 
    plt.xlabel("Effective Stellar Temp. [K]")
    plt.ylabel("Planets per 100 Stars")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

/var/folders/bb/byt860fx2d1_xjx67wpqq_wr0000gn/T/ipykernel_33813/2331110956.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  planetK2SubN["reli"]=reli*1
